In [1]:
#!/usr/bin/env python
import pika
import csv
import sys

connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()


In [13]:
import pika
import sys
import time
import csv
import pandas as pd
from datetime import datetime
import smtplib 

#load starting balances from Starting_Balance.csv
#input file
infile = 'Starting_Balance.csv'
infile_acct = "Account"
infile_sbal = "Starting Balance"

tran_df = pd.read_csv(infile, usecols = [infile_acct, infile_sbal])
print(tran_df)


for ind in tran_df.index: 
    if tran_df[infile_acct][ind] == 17938: 
        print(tran_df[infile_acct][ind], tran_df[infile_sbal][ind])



    Account Starting Balance
0     16549       12,532.88 
1     15169       15,000.00 
2     12053       20,000.00 
3     15022       50,000.00 
4     17938      100,000.00 
5     19202       13,000.00 
6     17622       15,000.00 
7     18789       20,000.00 
8     19532       50,000.00 
9     14447      100,000.00 
10    14736       80,000.00 
11    13699       70,000.00 
12    19548       90,000.00 
13    11737       20,000.00 
14    12594       25,000.00 
15    13894       30,000.00 
16    13202       20,000.00 
17    18637       10,000.00 
18    11066       60,000.00 
19    15191       30,000.00 
17938  100,000.00 


In [ ]:
import pika
import sys
import time
import csv
import pandas as pd
from datetime import datetime
import smtplib 


#load starting balances from Starting_Balance.csv
#input file
infile = 'Starting_Balance.csv'
infile_acct = "Account"
infile_sbal = "Starting Balance"

low_bal_amt = 100.00

tran_df = pd.read_csv(infile, usecols = [infile_acct, infile_sbal])
print(tran_df)


            

#set queue names
queue1 = "tran_queue"

#create lists
lst_tran_time_temp = []


# callback

# define a callback function to be called when a transaction message is received
def tran_callback(ch, method, properties, body):
    """ Define behavior on getting a message."""
    # decode the binary message body to a string
    print(f" [x] Transaction Received: {body.decode()}")
    # acknowledge the message was received and processed 
    # (now it can be deleted from the queue)
    #ch.basic_ack(delivery_tag=method.delivery_tag)

    #save/split trans 
    tran_current_str = body.decode()
    tran_current_list = tran_current_str.split(", ")
    tran_date = tran_current_list.split()[0]
    tran_acct = tran_current_list.split()[1]
    tran_amt = tran_current_list.split()[2]
    tran_merch = tran_current_list.split()[3]

    acct_bal = 0

    if tran_date != '' and tran_acct != '' and tran_amt != '':
        for ind in tran_df.index: 
            if tran_df[infile_acct][ind] == tran_acct: 
                acct_bal = tran_df[infile_sbal][ind]
                acct_bal = acct_bal - tran_amt
                #print(tran_df[infile_acct][ind], tran_df[infile_sbal][ind])

        if acct_bal <= low_bal_amt:
            if acct_bal < 0: 
                out_msg = "!!! Account Overdrawn!"
            else:
                out_msg = "!!  Low Balance Alert!"
        else:
            out_msg = ""

        if out_msg != "":
            print(f"{out_msg} - Acct:{tran_acct}, Tran Amt:{tran_amt}, Acct Bal:{acct_bal}")